# Task 2: Break XOR (Brute Force)

## Brute-Force Script

In [ ]:
import sys

def xor_decrypt(data, key):
    """
    # Performs single-byte XOR decryption.
    # Why XOR works for both encryption and decryption:
    # Because (P ⊕ K) ⊕ K = P (XOR is self-inverse).
    """
    try:
        return bytes([byte ^ key for byte in data])
    except:
        """
        # Possible exceptions:
        # - TypeError if key is not an integer
        # - Issues if data is not in bytes format
        """
        return None


def score_plaintext(candidate):
    """
    # Simple scoring function to determine if output resembles English text.
    # Why scoring is needed:
    # Because brute force generates 256 outputs — we must rank them.
    """

    if candidate is None:
        return -1

    printable = sum(1 for b in candidate if 32 <= b <= 126 or b in (9, 10, 13))
    letters = sum(1 for b in candidate if (65 <= b <= 90) or (97 <= b <= 122))
    spaces = candidate.count(32)

    score = printable + letters + (spaces * 2)
    return score


def brute_force_xor(ciphertext):
    """
    # Tries all 256 possible single-byte keys (0x00 - 0xFF).
    # Why 256 keys?
    # Because a single byte has 2^8 = 256 possible values.
    """

    results = []

    for key in range(256):
        decrypted = xor_decrypt(ciphertext, key)
        score = score_plaintext(decrypted)
        results.append((score, key, decrypted))

    # Sort from highest score to lowest
    results.sort(reverse=True, key=lambda x: x[0])

    return results


def main():
    """
    # Checks for correct command-line arguments.
    # Why use len(sys.argv)?
    # To ensure user provides required ciphertext filename.
    """

    if len(sys.argv) != 2:
        print("Usage: python3 bruteforce_xor.py <ciphertext_file>")
        sys.exit(1)

    filename = sys.argv[1]

    try:
        with open(filename, "rb") as f:
            ciphertext = f.read()
    except:
        print("Error: Could not open file.")
        sys.exit(1)

    print(f"Brute-forcing single-byte XOR on {filename}...")
    print("Trying all 256 possible keys...\n")

    results = brute_force_xor(ciphertext)

    print("Top 5 candidate keys:\n")

    for i in range(5):
        score, key, plaintext = results[i]
        print(f"{i+1}) Key: 0x{key:02x} | Score: {score}")
        print("Preview:", plaintext[:100].decode(errors="replace"))
        print()

    best_score, best_key, best_plaintext = results[0]

    with open("recovered_text.txt", "wb") as out:
        out.write(best_plaintext)

    print(f"[+] Recovered Key: 0x{best_key:02x}")
    print("[+] Decrypted plaintext saved to recovered_text.txt")


if __name__ == "__main__":
    main()


In [ ]:
import sys

def read_ciphertext(filename):
    """
    # Reads the ciphertext file in binary mode ("rb").
    # Why binary mode?
    # XOR operates on raw bytes. Text mode may corrupt data via decoding/newlines.
    """
    try:
        with open(filename, "rb") as f:
            return f.read()
    except:
        """
        # Possible errors:
        # - File not found
        # - Permission denied
        # - Wrong filename/path
        """
        return None
    
def xor_with_key(data, key):
    """
    # Applies single-byte XOR to a bytes object.
    # Why this works for decryption:
    # If C = P ⊕ K, then P = C ⊕ K because XOR is its own inverse.
    #
    # data: ciphertext bytes (or plaintext bytes)
    # key: integer 0..255 (single byte)
    # returns: bytes result after XOR
    """
    try:
        return bytes([b ^ key for b in data])
    except Exception as e:
        print(f"[!] XOR error: {e}")
        return None
    
def printable_ascii_ratio(candidate):
    """
    Returns fraction of bytes that are printable ASCII (32–126 only).
    """
    if candidate is None or len(candidate) == 0:
        return 0.0

    printable_count = 0

    for b in candidate:
        if 32 <= b <= 126:
            printable_count += 1

    return printable_count / len(candidate)

def letter_space_ratio(candidate):
    """
    Returns fraction of bytes that are letters (A-Z, a-z) or spaces.
    English text is dominated by letters and spaces.
    """
    if candidate is None or len(candidate) == 0:
        return 0.0

    count = 0

    for b in candidate:
        if (65 <= b <= 90) or (97 <= b <= 122) or b == 32:
            count += 1

    return count / len(candidate)

def common_word_score(candidate):
    """
    Counts occurrences of very common English words.
    More occurrences => more likely valid English plaintext.
    """
    if candidate is None or len(candidate) == 0:
        return 0

    lower = candidate.lower()

    common_words = [
        b" the ",
        b" and ",
        b" is ",
        b" to ",
        b" of ",
        b" in ",
        b" security ",
        b" xor "
    ]

    word_count = 0

    for word in common_words:
        word_count += lower.count(word)

    return word_count

def score_candidate(candidate):
    """
    Scores a candidate plaintext by how close its statistics are to expected English.
    Uses:
      - printable_ascii_ratio: target ~ 0.98
      - letter_space_ratio: target ~ 0.85 (midpoint of 0.80-0.90)

    Higher score = closer to expected English-like ratios.
    """

    pr = printable_ascii_ratio(candidate)
    lsr = letter_space_ratio(candidate)

    # Targets based on typical English text characteristics (with punctuation allowed)
    target_pr = 0.98
    target_lsr = 0.85

    # Distance from targets (smaller is better)
    dist_pr = abs(pr - target_pr)
    dist_lsr = abs(lsr - target_lsr)

    # Weighted distance
    weighted_distance = (1.0 * dist_pr) + (2.0 * dist_lsr)

    # Convert distance to positive closeness score
    score = 1.0 - weighted_distance

    # The closer to 1 the score is the better
    return score

def brute_force_xor(ciphertext):
    """
    Tries all keys 0x00..0xFF, decrypts with each key, computes score, and sorts.
    Returns a list of tuples: (score, key, candidate_plaintext) sorted best->worst.
    """
    results = []

    for key in range(256):
        candidate = xor_with_key(ciphertext, key)
        s = score_candidate(candidate)
        results.append((s, key, candidate))

    results.sort(reverse=True, key=lambda x: x[0])
    return results
    

def main():
    """
    # Validates command-line arguments using len(sys.argv).
    # Why?
    # The script needs exactly one input: the ciphertext filename.
    """
    if len(sys.argv) != 2:
        print("Usage: python3 bruteforce_xor.py <xor_chal_text.bin>")
        sys.exit(1)

    filename = sys.argv[1]
    ciphertext = read_ciphertext(filename)

    if ciphertext is None:
        print("[!] Error: Could not read the ciphertext file.")
        sys.exit(1)

    print(f"[*] Loaded ciphertext from: {filename}")
    print(f"[*] Ciphertext size: {len(ciphertext)} bytes")

    # Sanity test: XOR the first 16 bytes with an example key (0x00 keeps bytes unchanged)
    test_key = 0x00
    test_out = xor_with_key(ciphertext[:16], test_key)
    print(f"[*] Sanity test with key=0x{test_key:02x}: {test_out.hex()}")

    # Ciphertext tests
    print("\n--- Ciphertext Metrics ---")

    # Test printable ratio on ciphertext itself
    ratio = printable_ascii_ratio(ciphertext)
    print(f"[*] Printable ratio of ciphertext: {ratio:.4f}")

    # Test letter_space_ratio on ciphertext
    cipher_ratio = letter_space_ratio(ciphertext)
    print(f"[*] Letter+Space ratio of ciphertext: {cipher_ratio:.4f}")

    # Test common_word_score on ciphertext
    cipher_word_score = common_word_score(ciphertext)
    print(f"[*] Common word score of ciphertext: {cipher_word_score}")

    # English test sentence
    test_sentence = b"This is the security lab and the xor exercise."

    print("\n--- English Test Sentence Metrics ---")

    print(f"[*] Printable ratio: {printable_ascii_ratio(test_sentence):.4f}")
    print(f"[*] Letter+Space ratio: {letter_space_ratio(test_sentence):.4f}")
    print(f"[*] Common word score: {common_word_score(test_sentence)}")

    # Brute-force XOR decryption Scoring
    results = brute_force_xor(ciphertext)

    print("\n--- Top 3 candidate keys by Combination Scoring Rule ---\n")
    for i in range(3):
        s, key, candidate = results[i]
        preview = candidate[:120].decode(errors="replace")
        pr = printable_ascii_ratio(candidate)
        lsr = letter_space_ratio(candidate)
        print(f"{i+1}) Key: 0x{key:02x} | Score: {s:.6f} | printable={pr:.4f} | letter+space={lsr:.4f}")
        print(f"   Preview: {preview}\n")

    # Recover and save the best plaintext
    best_score, best_key, best_plaintext = results[0]

    print(f"[+] Selected best key: 0x{best_key:02x} (score={best_score:.6f})")

    output_filename = "recovered_text.txt"
    with open(output_filename, "wb") as out:
        out.write(best_plaintext)

    print(f"[+] Saved decrypted plaintext to: {output_filename}")

    # Quick verification metrics on recovered plaintext
    pr_best = printable_ascii_ratio(best_plaintext)
    lsr_best = letter_space_ratio(best_plaintext)

    print("\n--- Verification Metrics (Recovered Plaintext) ---")
    print(f"Printable ratio: {pr_best:.4f}")
    print(f"Letter+Space ratio: {lsr_best:.4f}")

if __name__ == "__main__":
    main()


[!] Error: Could not read the ciphertext file.


SystemExit: 1

## 5. Define a "Plausibility Test" (Scoring Rule)

Additional Thoughts: After building each individually I thought it best to combine "Printable ASCII ratio" and "Letter frequency". The rationale came down to both of these methods provide a good way to measure the potential candidacy of plaintexts and offering a way to cover one another while being a simiar method of testing. My biggest reason for excluding "Common word detection" is that within the current scope I cannot find a reasonable way to normalize the score for it to be worth using. Common words and the percentage of them within a sentence can vary greatly depending on context and topic so having a metric so reliant on this seems unwise. What would be better much like proper brute force methods is have a large dictionary of words (in the case of cracking, top 100,000 most common passwords) to search against and score for how high a ratio of matching words there are, having only common words seems to unreliable. Meanwhile for printable ASCII we can say a ratio as close to 1.0 would be good for english and letter+space would be 0.80-0.90 as again this is the metric expected for the English langauage in writting.

1. Single-byte XOR encryption is extremely vulnerable to brute-force attacks because the range of keys is simply too small. At a single-byte XOR encryption we have a range of 2^8 (0x00 – 0xFF = 256 possible keys) which is small enough to try all keys within a second with most modern computers. So simply try every key.

2. As previously mentioned the total range of keys for single-byte is 0x00-0xFF which is 2^8 = 256 total possible keys.

3. In the brute_force_xor function we run a for loop in range of 256 which allows us to generate and apply each of the keys to the cipher text. Each key is applied to the cipher with a xor with the xor_with_key function. The result of the xor is evaluated with the Scoring Rule that is outlined above.

4. Scoring is necessary since we need a way to be able to effectively parse each of the 256 results because manually checking this many for a human would be prone to error and brain numbing. Meanwhile using a scoring system to find patterns common in the English language would be fairly easy for a machine to do.

5. English plaintext has certain patterns that make it substantially different compared to random plaintext. Of note are the two methods I decided to go with which were "Printable ASCII ratio" and "Letter frequency". The English language consists of only printable ASCII so if something shows up outside of this we know something is wrong (scores closer to 1 is better but 0.98 to account for indents and newlines). Second the English language consists of 80-90 percent letters and spaces, characters outside of this are only a small ratio in normal english so we want to get a value as close to this range as possible (aiming for 0.85) for simplicity. So a combination of these two scores should closely result in only producing the highest score for something that would closely resemble English which would otherwise be highly unlikely to be the case, especially with the small sample size of 256. This method may fail at larger scales but then may benefit from a dictionary approach outlined in the explaination above.

6. Random outputs may produce high scores purely out of luck, it is just odds. Since we are not explictly scoring exact English words, this is an outcome that can be expected and more likely with a larger sample size.

7. Key 0x1d is the top key and it was the correct result. It scored the same as the other top contenders in printable but it scored way higher in the letter+space which is weighted higher since it is a better English metric. We can see it is correct since the preview shows that it is clearly English writting.

    --- Top 3 candidate keys by Combination Scoring Rule ---
    1) Key: 0x1d | Score: 0.941354 | printable=0.9714 | letter+space=0.8750
    Preview: CSCI/CSCY 4407 - Security & Cryptography (Spring 2026)
    Lab 2: XOR Brute Force Challenge

    If you can read this message cl

    2) Key: 0x1c | Score: 0.661146 | printable=0.9714 | letter+space=0.6849
    Preview: BRBH.BRBX!5516!,!Rdbtshux!'!Bsxqunfs`qix!)Rqshof!3137(
                                                                    M`c!3;!YNS!Cstud!Gnsbd!Bi`mmdofd

                                                                                                    Hg!xnt!b`o!sd`e!uihr!ldrr`fd!bm

    3) Key: 0x1f | Score: 0.661146 | printable=0.9714 | letter+space=0.6849
    Preview: AQAK-AQA["6625"/"Qgawpkv{"$"Ap{rvmepcrj{"*Qrpkle"0204Nc`"08"ZMP"@pwvg"Dmpag"AjcnnglKd"{mw"acl"pgcf"vjkq"ogqqceg"an

8. Applying the xor_tool.py to the xor_chal_text.bin with the correct key does result in the same decrypted text file. This would be expected as we know the key so using the previous xor script would provide the same output. "python3 xor_tool.py decrypt xor_chal_text.bin task1_decrypted.txt 1D" was used.

9. High key entropy is crucial in cryptograph systems. It is painfully highlighted in this lab as with a single-byte xor encryption we were able to brute force crack in less than a second. Meanwhile with proper key entropy it makes brute force cracking unfeasible, either in the time that it would take or the cost for the amount of resources to be able to crack in any kind of reasonable time. It is important to have key entropy that produces an encryption that takes more than its worth to crack. Single-byte is simply not anywhere enough for real life applications.

10. For 16 byte key it would then be 16x8=128 bits, then the amount of possible keys would be 2^128 which is equal to 3.4~ e+38. This is an amount of keys which is used by most modern symmetric encryption because to brute force at a rate of a billion keys per second would result in it taking longer than the heat death of the universe. So this would be computationally infeasibile

11. No. None of the scoring methods discussed would be able to work properly on binary. It would pass on the printable ASCII but fail on the letter+space. In binary we would not be able to rate it the same as we would for the patterns found in normal characters for English. For this to work we would need to be able to define patterns found in binary that produces English or simply translate the compressed binary into characters but this feels against the heart of the question.

12. While at a glance applying a random encryption to a text file to protect it, unless you understand what type of encryption is being used and why it may just be trival work for a savy actor. The two major parts of note is understanding that XOR encryption is reversible which has potential risk since it uses the same key for encryption and decryption. This would not be an issue if the key is kept secret OR if the range of keys used it not large enough, which was the case here. A single-byte XOR encryption offers no protection against brute force cracking since its key entropy is extremely low.